# Import Libraries

In [ ]:
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
import os
from dotenv import load_dotenv
load_dotenv()

# Define Tools

In [ ]:
index_name: str = "demo"
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddings",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def doc_search(question: str) -> str:
    "Search documents"
    docs = vector_store.similarity_search(
    query=question,
    k=3,
    search_type="hybrid",
    )
    return docs


tools = [multiply, add, doc_search]

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()
# Choose the LLM that will drive the agent
# Only certain models support this
model = AzureChatOpenAI(api_key=os.getenv("AZURE_OPENAI_API_KEY"), azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"), deployment_name="gpt35_turbo", api_version="2024-03-01-preview")

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "add 110 to 200 and multiply the result by 2"
    }
)

In [ ]:
agent_executor.invoke(
    {
        "input": "search docs and tell me Who was this guide developed for?"
    }
)